In [1]:
import pandas as pd

In [2]:
#this is alist so there is need for us to coonvert it into a dataframe
df_world = pd.read_csv('world_revenue.csv')

In [3]:
#converting my data to a dataframe
df = pd.DataFrame(df_world)

In [4]:
df.to_csv("world_largest_companies_updated.csv", index=False)

In [5]:
df = pd.read_csv("world_largest_companies_updated.csv", sep=',')

In [6]:
df.head()

,Rank,Name,Industry,Revenue (USD millions),Revenue growth,Employees,Headquarters
0,1,Walmart,Retail,611289,6.7%,2100000,"Bentonville, Arkansas"
1,2,Amazon,Retail and cloud computing,513983,9.4%,1540000,"Seattle, Washington"
2,3,ExxonMobil,Petroleum industry,413680,44.8%,62000,"Spring, Texas"
3,4,Apple,Electronics industry,394328,7.8%,164000,"Cupertino, California"
4,5,UnitedHealth Group,Healthcare,324162,12.7%,400000,"Minnetonka, Minnesota"


In [7]:
#drop unnecessary column
#to_drop = ['Headquarters']

#df.drop(to_drop, inplace=True, axis=1)

In [8]:
df.head()

,Rank,Name,Industry,Revenue (USD millions),Revenue growth,Employees,Headquarters
0,1,Walmart,Retail,611289,6.7%,2100000,"Bentonville, Arkansas"
1,2,Amazon,Retail and cloud computing,513983,9.4%,1540000,"Seattle, Washington"
2,3,ExxonMobil,Petroleum industry,413680,44.8%,62000,"Spring, Texas"
3,4,Apple,Electronics industry,394328,7.8%,164000,"Cupertino, California"
4,5,UnitedHealth Group,Healthcare,324162,12.7%,400000,"Minnetonka, Minnesota"


In [9]:
#dropping the index
df.reset_index(drop=True, inplace=True)

#if you were to set the index
#df = df.set_index('identifier')

In [10]:
df.head()

,Rank,Name,Industry,Revenue (USD millions),Revenue growth,Employees,Headquarters
0,1,Walmart,Retail,611289,6.7%,2100000,"Bentonville, Arkansas"
1,2,Amazon,Retail and cloud computing,513983,9.4%,1540000,"Seattle, Washington"
2,3,ExxonMobil,Petroleum industry,413680,44.8%,62000,"Spring, Texas"
3,4,Apple,Electronics industry,394328,7.8%,164000,"Cupertino, California"
4,5,UnitedHealth Group,Healthcare,324162,12.7%,400000,"Minnetonka, Minnesota"


In [11]:
#lowercase all the columns and replace spaces with underscores
df.columns = df.columns.str.lower().str.replace(" ", "_")


In [12]:
df.dtypes

rank                       int64
name                      object
industry                  object
revenue_(usd_millions)     int64
revenue_growth            object
employees                 object
headquarters              object
dtype: object

In [13]:
string_columns = list(df.dtypes[df.dtypes == 'object'].index)

#also lowercasing and replacing all spaces with underscores for string columns
for col in string_columns:
    df[col] = df[col].str.lower().replace(" ", "")

In [14]:
df.head()

,rank,name,industry,revenue_(usd_millions),revenue_growth,employees,headquarters
0,1,walmart,retail,611289,6.7%,2100000,"bentonville, arkansas"
1,2,amazon,retail and cloud computing,513983,9.4%,1540000,"seattle, washington"
2,3,exxonmobil,petroleum industry,413680,44.8%,62000,"spring, texas"
3,4,apple,electronics industry,394328,7.8%,164000,"cupertino, california"
4,5,unitedhealth group,healthcare,324162,12.7%,400000,"minnetonka, minnesota"


In [15]:
#writing back to csv after cleaning
df.to_csv("new_worlds_largest_companies.csv", index=False)

In [16]:
df = pd.read_csv("new_worlds_largest_companies.csv", sep=',')

In [17]:
df.head()

,rank,name,industry,revenue_(usd_millions),revenue_growth,employees,headquarters
0,1,walmart,retail,611289,6.7%,2100000,"bentonville, arkansas"
1,2,amazon,retail and cloud computing,513983,9.4%,1540000,"seattle, washington"
2,3,exxonmobil,petroleum industry,413680,44.8%,62000,"spring, texas"
3,4,apple,electronics industry,394328,7.8%,164000,"cupertino, california"
4,5,unitedhealth group,healthcare,324162,12.7%,400000,"minnetonka, minnesota"


In [18]:
from pyspark.sql import SparkSession
from pyspark.sql import types
from pyspark.conf import SparkConf

In [19]:
#setting up the postgresjar
conf = SparkConf()
conf.set('spark.jars', 'postgresql-42.7.3.jar')

In [20]:
spark = SparkSession.builder \
         .config(conf=conf) \
         .master("local[*]") \
         .appName("pyspark data loadng") \
         .config("spark.executor.memory", "4g") \
         .config("spark.executor.cores", "3") \
         .config("spark.cores.max", "3") \
         .enableHiveSupport() \
         .getOrCreate()

24/05/10 12:55:24 WARN Utils: Your hostname, DESKTOP-R4B4RHT resolves to a loopback address: 127.0.1.1; using 172.30.252.130 instead (on interface eth0)
24/05/10 12:55:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/05/10 12:55:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [21]:
spark_df = spark.createDataFrame(df)

In [22]:
spark_df.show()

+----+--------------------+--------------------+----------------------+--------------+---------+--------------------+
|rank|                name|            industry|revenue_(usd_millions)|revenue_growth|employees|        headquarters|
+----+--------------------+--------------------+----------------------+--------------+---------+--------------------+
|   1|             walmart|              retail|                611289|          6.7%|  2100000|bentonville, arka...|
|   2|              amazon|retail and cloud ...|                513983|          9.4%|  1540000| seattle, washington|
|   3|          exxonmobil|  petroleum industry|                413680|         44.8%|    62000|       spring, texas|
|   4|               apple|electronics industry|                394328|          7.8%|   164000|cupertino, califo...|
|   5|  unitedhealth group|          healthcare|                324162|         12.7%|   400000|minnetonka, minne...|
|   6|          cvs health|          healthcare|        

In [23]:
spark_df.schema

StructType([StructField('rank', LongType(), True), StructField('name', StringType(), True), StructField('industry', StringType(), True), StructField('revenue_(usd_millions)', LongType(), True), StructField('revenue_growth', StringType(), True), StructField('employees', StringType(), True), StructField('headquarters', StringType(), True)])

In [24]:
spark_df.printSchema()

root
 |-- rank: long (nullable = true)
 |-- name: string (nullable = true)
 |-- industry: string (nullable = true)
 |-- revenue_(usd_millions): long (nullable = true)
 |-- revenue_growth: string (nullable = true)
 |-- employees: string (nullable = true)
 |-- headquarters: string (nullable = true)



In [25]:
schema = types.StructType([
    types.StructField('rank', types.IntegerType(), True), 
    types.StructField('name', types.StringType(), True), 
    types.StructField('industry', types.StringType(), True), 
    types.StructField('revenue_(usd_millions)',types.IntegerType(), True), 
    types.StructField('revenue_growth', types.StringType(), True), 
    types.StructField('employees', types.IntegerType(), True), 
    types.StructField('headquarters', types.StringType(), True)
])


In [26]:
spark_df =spark.read.options(header = "false", sep=',').schema(schema).csv("new_worlds_largest_companies.csv")

In [27]:
spark_df.show()

+----+--------------------+--------------------+----------------------+--------------+---------+--------------------+
|rank|                name|            industry|revenue_(usd_millions)|revenue_growth|employees|        headquarters|
+----+--------------------+--------------------+----------------------+--------------+---------+--------------------+
|NULL|                name|            industry|                  NULL|revenue_growth|     NULL|        headquarters|
|   1|             walmart|              retail|                611289|          6.7%|  2100000|bentonville, arka...|
|   2|              amazon|retail and cloud ...|                513983|          9.4%|  1540000| seattle, washington|
|   3|          exxonmobil|  petroleum industry|                413680|         44.8%|    62000|       spring, texas|
|   4|               apple|electronics industry|                394328|          7.8%|   164000|cupertino, califo...|
|   5|  unitedhealth group|          healthcare|        

In [28]:
spark_df.schema

StructType([StructField('rank', IntegerType(), True), StructField('name', StringType(), True), StructField('industry', StringType(), True), StructField('revenue_(usd_millions)', IntegerType(), True), StructField('revenue_growth', StringType(), True), StructField('employees', IntegerType(), True), StructField('headquarters', StringType(), True)])

In [29]:
spark_df.printSchema()

root
 |-- rank: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- industry: string (nullable = true)
 |-- revenue_(usd_millions): integer (nullable = true)
 |-- revenue_growth: string (nullable = true)
 |-- employees: integer (nullable = true)
 |-- headquarters: string (nullable = true)



In [30]:
#write the data to our postgres
if spark_df:
    try:
         spark_df.write.format("jdbc") \
                 .options(url="jdbc:postgresql://172.21.0.3:5432/postgres",
                          dbtable="public.world_largest_companies",
                          user="root",
                          password="root",
                          driver="org.postgresql.Driver") \
                 .mode('overwrite') \
                 .save()
    except Exception as e:
        print("excuse me only spark dataframes are allowed")
   
   

24/05/10 12:55:39 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
